In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/Users/huiw/.virtualenvs/cv/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
#Problem 1
#Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to 
#adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor t 
#using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.

In [6]:
# logistic regression with l2 loss function regularizing with beta = 0.01

train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    l2_loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2_loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, l2_loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 45.922104
Training accuracy: 8.8%
Validation accuracy: 12.4%
Loss at step 100: 11.663520
Training accuracy: 73.1%
Validation accuracy: 72.1%
Loss at step 200: 4.425245
Training accuracy: 79.0%
Validation accuracy: 77.1%
Loss at step 300: 1.965915
Training accuracy: 82.0%
Validation accuracy: 80.0%
Loss at step 400: 1.130253
Training accuracy: 83.6%
Validation accuracy: 81.5%
Loss at step 500: 0.843313
Training accuracy: 84.0%
Validation accuracy: 81.9%
Loss at step 600: 0.743699
Training accuracy: 84.3%
Validation accuracy: 82.2%
Loss at step 700: 0.708816
Training accuracy: 84.3%
Validation accuracy: 82.2%
Loss at step 800: 0.696517
Training accuracy: 84.4%
Validation accuracy: 82.3%
Test accuracy: 88.4%


In [8]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    l2_loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2_loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, l2_loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.875362
Minibatch accuracy: 9.4%
Validation accuracy: 13.5%
Minibatch loss at step 500: 0.901935
Minibatch accuracy: 82.0%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 0.925165
Minibatch accuracy: 78.1%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 0.968390
Minibatch accuracy: 76.6%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.778080
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 0.887294
Minibatch accuracy: 75.8%
Validation accuracy: 80.4%
Minibatch loss at step 3000: 0.714614
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Test accuracy: 87.7%


In [21]:
## neural network with l2 loss function

batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    l2_loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2_loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases) 
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, l2_loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 49.536858
Minibatch accuracy: 13.3%
Validation accuracy: 14.5%
Minibatch loss at step 500: 0.881918
Minibatch accuracy: 83.6%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 0.925464
Minibatch accuracy: 78.1%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 0.968345
Minibatch accuracy: 76.6%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.778038
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 0.887300
Minibatch accuracy: 75.8%
Validation accuracy: 80.4%
Minibatch loss at step 3000: 0.714609
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Test accuracy: 87.7%


In [10]:
## neural network(one hidden layer) with l2 loss function

batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(hidden_layer, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    l2_loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2_loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu(tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, l2_loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 367.567749
Minibatch accuracy: 8.6%
Validation accuracy: 33.9%
Minibatch loss at step 500: 22.552784
Minibatch accuracy: 78.9%
Validation accuracy: 72.0%
Minibatch loss at step 1000: 6.304079
Minibatch accuracy: 75.8%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 25.410654
Minibatch accuracy: 69.5%
Validation accuracy: 71.7%
Minibatch loss at step 2000: 5.085789
Minibatch accuracy: 84.4%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 10.868015
Minibatch accuracy: 71.1%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 4.046955
Minibatch accuracy: 84.4%
Validation accuracy: 79.6%
Test accuracy: 86.2%


In [ ]:
#Problem 2
#Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [12]:
#log reg with l2 loss and small training data

train_subset = 100
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    l2_loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2_loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [13]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, l2_loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 49.163628
Training accuracy: 11.0%
Validation accuracy: 9.0%
Loss at step 100: 11.178214
Training accuracy: 100.0%
Validation accuracy: 47.2%
Loss at step 200: 4.165465
Training accuracy: 100.0%
Validation accuracy: 55.0%
Loss at step 300: 1.602042
Training accuracy: 100.0%
Validation accuracy: 61.8%
Loss at step 400: 0.666850
Training accuracy: 100.0%
Validation accuracy: 65.9%
Loss at step 500: 0.325441
Training accuracy: 100.0%
Validation accuracy: 67.9%
Loss at step 600: 0.200491
Training accuracy: 100.0%
Validation accuracy: 68.5%
Loss at step 700: 0.154553
Training accuracy: 100.0%
Validation accuracy: 68.7%
Loss at step 800: 0.137506
Training accuracy: 100.0%
Validation accuracy: 68.6%
Test accuracy: 75.3%


In [14]:
#neural network wiht l2 loss and small training data

batch_size = 12
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(hidden_layer, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    l2_loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2_loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu(tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [15]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, l2_loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 485.429718
Minibatch accuracy: 8.3%
Validation accuracy: 17.1%
Minibatch loss at step 500: 648.380981
Minibatch accuracy: 58.3%
Validation accuracy: 60.2%
Minibatch loss at step 1000: 44.162453
Minibatch accuracy: 58.3%
Validation accuracy: 49.8%
Minibatch loss at step 1500: 64.770660
Minibatch accuracy: 41.7%
Validation accuracy: 46.7%
Minibatch loss at step 2000: 15.400859
Minibatch accuracy: 50.0%
Validation accuracy: 40.3%
Minibatch loss at step 2500: 15.435740
Minibatch accuracy: 33.3%
Validation accuracy: 44.9%
Minibatch loss at step 3000: 12.477357
Minibatch accuracy: 58.3%
Validation accuracy: 49.9%
Test accuracy: 55.8%


In [16]:
#Problem 3
#Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during 
#training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides 
#nn.dropout() for that, but you have to make sure it's only inserted during training.

#What happens to our extreme overfitting case?

In [17]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    l2_loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2_loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu(tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, l2_loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 521.423828
Minibatch accuracy: 10.9%
Validation accuracy: 35.6%
Minibatch loss at step 500: 29.331917
Minibatch accuracy: 75.0%
Validation accuracy: 72.5%
Minibatch loss at step 1000: 23.384016
Minibatch accuracy: 68.0%
Validation accuracy: 75.4%
Minibatch loss at step 1500: 53.866127
Minibatch accuracy: 56.2%
Validation accuracy: 74.8%
Minibatch loss at step 2000: 9.690140
Minibatch accuracy: 72.7%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 10.877956
Minibatch accuracy: 65.6%
Validation accuracy: 79.3%
Minibatch loss at step 3000: 3.955190
Minibatch accuracy: 78.1%
Validation accuracy: 78.8%
Test accuracy: 84.8%


In [ ]:
#Problem 4
#Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep 
#network is 97.1%.

#One avenue you can explore is to add multiple layers.

#Another one is to use learning rate decay:

#global_step = tf.Variable(0)  # count the number of steps taken.
#learning_rate = tf.train.exponential_decay(0.5, step, ...)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [23]:
#trying smaller beta and bigger num steps

batch_size = 128
beta = 0.0001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    l2_loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2_loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu(tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [24]:
num_steps = 6001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, l2_loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 500.518707
Minibatch accuracy: 13.3%
Validation accuracy: 44.5%
Minibatch loss at step 500: 43.292088
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 18.358690
Minibatch accuracy: 68.0%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 45.968636
Minibatch accuracy: 67.2%
Validation accuracy: 76.1%
Minibatch loss at step 2000: 9.933962
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 2500: 15.854102
Minibatch accuracy: 64.8%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 5.119968
Minibatch accuracy: 73.4%
Validation accuracy: 80.3%
Minibatch loss at step 3500: 4.307816
Minibatch accuracy: 82.0%
Validation accuracy: 80.1%
Minibatch loss at step 4000: 4.090047
Minibatch accuracy: 74.2%
Validation accuracy: 80.3%
Minibatch loss at step 4500: 4.227316
Minibatch accuracy: 76.6%
Validation accuracy: 79.6%
Minibatch loss at step 5000: 5.886736
Minibatch accuracy: 75.8%
Validation a